# <center> 流畅的python </center>
## 第十一章 接口：从协议到抽象基类
**示例11-1 vector2d (v0)：x, y是公开数据属性**

In [1]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))

**示例11-2 Vector2d (v3) 使用特性实现x, y**

In [2]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))

**示例11-3 定义getitem方法**

In [3]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]
    
f = Foo()
print(f[1])
for i in f:
    print(i)

10
0
10
20


**示例11-4 实现序列协议的FrenchDeck类**

In [4]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck():
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = "spades diamonds clubs hearts".split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]



**示例11-5 random.shuffle函数不能打乱FrenchDeck实例**

In [5]:
from random import shuffle

deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

**示例11-6 为FrenchDeck打猴子补丁，把它变成可变的，让random.shuffle函数能处理**

In [6]:
def set_card(deck, position, card):
    deck._cards[position] = card
    
FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='4', suit='hearts'),
 Card(rank='8', suit='hearts'),
 Card(rank='3', suit='hearts'),
 Card(rank='7', suit='diamonds'),
 Card(rank='10', suit='diamonds')]

**示例11-7 使用鸭子类型处理单个字符或由字符串组成的可迭代对象**

In [7]:
try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass
field_names = tuple(field_names)

NameError: name 'field_names' is not defined

**示例11-8 FrenchDeck2, collections.MutableSequence的子类**

In [8]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = "spades diamonds clubs hearts".split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del(self._cards[position])
        
    def insert(self, position, value):
        self._cards.insert(position, value)

**示例11-9 Tombola是抽象基类，有两个抽象方法和一个具体方法**

In [16]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        '''从可迭代对象中添加元素'''
        
    @abc.abstractmethod
    def pick(self):
        '''随机删除元素，然后将其返回。如果示例为空，这个方法应该抛出LookupError'''
        
    def loaded(self):
        '''如果至少有一个元素，返回True, 否则返回False。'''
        return bool(self.inspect())
    
    def inspect(self):
        '''返回一个有序元组，有当前元素组成'''
        items = []
        while(True):
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

**示例11-11 不符合Tombola要求的子类无法蒙混过关**

In [17]:
class Fake(Tombola):
    def pick(self):
        return 13
    
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

**示例11-12 定义Tombola的具体子类**

In [18]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

**示例11-13 Tombola的具体子类LotteryBlower**

In [19]:
class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

**示例11-14 TomboList是Tombola的虚拟子类**

In [20]:
from random import randrange

@Tombola.register
class TomboList(list):
    
    def pick(self):
        if(self):
            position = eandrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
            
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    